# 数据准备

## 数据预处理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format='retina'

In [2]:
from help import read_data, data_preprocess
train, test, store = read_data()
X1, y1, X2 = data_preprocess(train, test, store)

Loading train data...


//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
//anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Loading test data...
Loading test data...
  Successfully read data as: train, test, store.

Preprocessing training data...
  Successfully preprocess training data.

Preprocessing testing data...
  Successfully preprocess testing data.



## 数据划分

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1)

# 模型

## 多元回归

In [4]:
from sklearn.linear_model import LinearRegression
Linear_Model = LinearRegression()
Linear_Model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [5]:
from help import rmspe
y_train_hat = Linear_Model.predict(X_train)
y_test_hat = Linear_Model.predict(X_test)
print(rmspe(y_train_hat, y_train))
print(rmspe(y_test_hat, y_test))

0.4709931692991407
0.46501034600974694
